# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 48 new papers today
          18 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/17 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.07252


extracting tarball to tmp_2210.07252...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.07258


extracting tarball to tmp_2210.07258...

...

extracting tarball to tmp_2210.07925...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.07933


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.07925/natnotes.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2210.07933...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.07258-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.07258) | **A Transient `Changing-Look' AGN Resolved on Month Timescales From  First-Year SDSS-V Data**  |
|| Grisha Zeltyn, et al. -- incl., <mark>Lorena Hernandez-Garcia</mark>, <mark>Paul Green</mark>, <mark>Muryel Guolo</mark>, <mark>Xin Liu</mark>, <mark>Joel R. Brownstein</mark>, <mark>Hector Ibarra-Medel</mark> |
|*Appeared on*| *2022-10-17*|
|*Comments*| *Resubmitted to ApJL after addressing the referee report*|
|**Abstract**| We report the discovery of a new `changing-look' active galactic nucleus (CLAGN) event, in the quasar SDSS J162829.17+432948.5 at z=0.2603, identified through repeat spectroscopy from the fifth Sloan Digital Sky Survey (SDSS-V). Optical photometry taken during 2020-2021 shows a dramatic dimming of ${\Delta}$g${\approx}$1 mag, followed by a rapid recovery on a timescale of several months, with the ${\lesssim}$2 month period of rebrightening captured in new SDSS-V and Las Cumbres Observatory spectroscopy. This is one of the fastest CLAGN transitions observed to date. Archival observations suggest that the object experienced a much more gradual dimming over the period of 2011-2013. Our spectroscopy shows that the photometric changes were accompanied by dramatic variations in the quasar-like continuum and broad-line emission. The excellent agreement between the pre- and post-dip photometric and spectroscopic appearances of the source, as well as the fact that the dimmest spectra can be reproduced by applying a single extinction law to the brighter spectral states, favor a variable line-of-sight obscuration as the driver of the observed transitions. Such an interpretation faces several theoretical challenges, and thus an alternative accretion-driven scenario cannot be excluded. The recent events observed in this quasar highlight the importance of spectroscopic monitoring of large AGN samples on weeks-to-months timescales, which the SDSS-V is designed to achieve. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.07542-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.07542) | **A Blueprint for the Milky Way's Stellar Populations. IV. Spectrum-based  Empirical Corrections and the Galactic Starburst Sequence**  |
|| Deokkeun An, Timothy C. Beers, <mark>Young Sun Lee</mark>, Thomas Masseron |
|*Appeared on*| *2022-10-17*|
|*Comments*| *34 pages, 25 figures, submitted to the AAS Journals*|
|**Abstract**| We continue our series of papers on phase-space distributions of stars in the Milky Way based on photometrically derived metallicities and Gaia astrometry, with a focus on the halo-disk interface in the local volume. To exploit various photometric databases, we develop a method of empirically calibrating synthetic stellar spectra based on a comparison with observations of stellar sequences and individual stars in SDSS, SMSS, and PS1, overcoming band-specific corrections employed in our previous work. In addition, photometric zero-point corrections are derived to provide an internally consistent photometric system with a spatially uniform metallicity zero point. Based on our phase-space diagrams, we find a strikingly narrow sequence in the rotational velocity ($v_\phi$) versus metallicity ([Fe/H]) space for a sample of high proper-motion stars ($>25$ mas yr$^{-1}$), which runs along the Gaia Sausage/Enceladus (GSE) and the Splash sub-structures, and is connected to the disk. Notably, a rapid increase of $v_\phi$ from a nearly zero net rotation to $200$ km s$^{-1}$ within a narrow metallicity interval ($-0.8 < {\rm [Fe/H]} < -0.2$) suggests that these stars were formed on a short gas-depletion time scale. Based on measurements of a scale height and length, we argue that they are distinct from stars dynamically heated by mergers, and are the relics formed during the starburst when the young Milky Way encountered the gas-rich GSE merger. The chain of high proper-motion stars, which we dub the Galactic Starburst Sequence (GSS), provides evidence that the post-merger metal-enriched gas settled onto the disk. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.07725-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.07725) | **A radio-detected thermonuclear supernova from a single-degenerate  progenitor with a helium star donor**  |
|| Erik C. Kool, et al. -- incl., <mark>Chelsea Harris</mark>, <mark>Miguel Pérez-Torres</mark>, <mark>Nora Linn Strotjohann</mark>, <mark>Mathew Smith</mark>, <mark>Frank Masci</mark>, <mark>Yashvi Sharma</mark> |
|*Appeared on*| *2022-10-17*|
|*Comments*| *52 pages, 9 figures, 6 tables. Under review*|
|**Abstract**| Type Ia supernovae are thought to be carbon-oxygen white dwarf stars that explode after accreting material from a companion star, but despite extensive studies the nature of the companion star is still poorly understood, as is the explosion mechanism. In the single degenerate scenario, the companion is a non-degenerate star that loses material through winds and/or binary interaction, and a few Type Ia supernovae have shown evidence for hydrogen-rich circumstellar material. We present here the study of SN 2020eyj, a unique Type Ia supernova showing delayed interaction with helium-rich, but hydrogen-poor, circumstellar material. This material surrounding SN 2020eyj is revealed by its unusual light curve and infrared emission, narrow helium emission lines and, for the first time ever in a Type Ia supernova, also a radio counterpart. The circumstellar material likely originates from the companion star, providing the first direct evidence for a, so far hypothesized, single degenerate progenitor system composed of a white dwarf and a helium donor star. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.07801-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.07801) | **A combinatorial algebraic approach for the modified second-generation  time-delay interferometry**  |
|| <mark>Zhang-Qi Wu</mark>, Pan-Pan Wang, <mark>Wei-Liang Qian</mark>, Cheng-Gang Shao |
|*Appeared on*| *2022-10-17*|
|*Comments*| **|
|**Abstract**| We generalize the combinatorial algebraic approach first proposed by Dhurandhar et al. to construct various classes of modified second-generation time-delay interferometry (TDI) solutions. The main idea behind the algorithm is to enumerate, in a given order, a specific type of commutator between two monomials defined by the products of particular time-displacement operators. On the one hand, the above commutators can be systematically rewritten as the elements of a left ideal, defined by the l.h.s. of the relevant equation for the TDI solution. On the other hand, these commutators are shown to vanish if we only keep up the first-order contributions regarding the rate of change of armlengths. In other words, each commutator furnishes a valid TDI solution pertaining to the given type of modified second-generation combinations. In this work, the original algorithm, which only involves time-delay operators, is extended by introducing the time-advance ones and then utilized to seek solutions of the Beacon, Relay, Monitor, Sagnac, and fully symmetric Sagnac types. We discuss the relation between the present scheme's solutions and those obtained by the geometric TDI approach, a well-known method of exhaustion of virtual optical paths. In particular, we report the results on novel Sagnac-inspired solutions that cannot be straightforwardly obtained using the geometric TDI algorithm. The average response functions, floor noise power spectral densities, and sensitivity functions are evaluated for the obtained solutions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.07925-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.07925) | **300: An ACA 870 $μ$m Continuum Survey of Orion Protostars and their  Evolution**  |
|| Samuel Federman, et al. -- incl., <mark>William J. Fischer</mark>, <mark>Lee Hartmann</mark> |
|*Appeared on*| *2022-10-17*|
|*Comments*| **|
|**Abstract**| We present an 870 $\mu$m continuum survey of 300 protostars from the Herschel Orion Protostar Survey (HOPS) using the Atacama Compact Array (ACA). These data measure protostellar flux densities on envelope scales $\le$ 8000 AU (20") and resolve the structure of envelopes with 1600 AU (4") resolution, a factor of 3-5 improvement in angular resolution over existing single dish 870 $\mu$m observations. We compare the ACA observations to Atacama Large Millimeter Array (ALMA) 12m array observations at 870 $\mu$m with $\sim$0.1" (40 AU) resolution. Using the 12 meter data to measure the fluxes from disks and the ACA fluxes within 2500 au to measure the combined disk plus envelope fluxes, we calculate the 12 m/ACA 870 $\mu$m flux ratios. Our sample shows a clear evolution in this ratio. Class 0 protostars are mostly envelope-dominated with ratios $<$ 0.5. In contrast, Flat Spectrum protostars are primarily disk-dominated with ratios near one, although with a number of face-on protostars with large envelopes. Class I protostars span the range from envelope to disk dominated. The increase in ratio is accompanied by a decrease in the envelope fluxes and estimated mass infall rates. We estimate that 80% of the mass is accreted during the envelope-dominated phase. We find that the 12m/ACA flux ratio is an evolutionary indicator that largely avoids the inclination and foreground extinction dependence of SED-based indicators. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.07933-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.07933) | **TOI-179: a young system with a transiting compact Neptune-mass planet  and a low-mass companion in outer orbit**  |
|| S. Desidera, et al. -- incl., <mark>A. Maggio</mark> |
|*Appeared on*| *2022-10-17*|
|*Comments*| *25 pages, 24 figures, A&A, in press*|
|**Abstract**| Transiting planets around young stars are key benchmarks for our understanding of planetary systems. One of such candidates was identified around the K dwarf HD 18599 by TESS, labeled as TOI-179. We present the confirmation of the transiting planet and the characterization of the host star and of the TOI-179 system over a broad range of angular separations. To this aim, we exploited the TESS photometric time series, intensive radial velocity monitoring performed with HARPS, and deep high-contrast imaging observations obtained with SPHERE and NACO at VLT. The inclusion of Gaussian processes regression analysis is effective to properly model the magnetic activity of the star and identify the Keplerian signature of the transiting planet. The star, with an age of 400+-100 Myr, is orbited by a transiting planet with period 4.137436 days, mass 24+-7 Mearth, radius 2.62 (+0.15-0.12) Rearth, and significant eccentricity (0.34 (+0.07-0.09)). Adaptive optics observations identified a low-mass companion at the boundary between brown dwarfs and very low mass stars (mass derived from luminosity 83 (+4-6) Mjup) at a very small projected separation (84.5 mas, 3.3 au at the distance of the star). Coupling the imaging detection with the long-term radial velocity trend and the astrometric signature, we constrained the orbit of the low mass companion, identifying two families of possible orbital solutions. The TOI-179 system represents a high-merit laboratory for our understanding of the physical evolution of planets and other low-mass objects and of how the planet properties are influenced by dynamical effects and interactions with the parent star. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.07919-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.07919) | **A model for the infrared-radio correlation of main-sequence galaxies at  GHz frequencies and its dependence on redshift and stellar mass**  |
|| J. Schober, M. T. Sargent, R. S. Klessen, <mark>D. R. G. Schleicher</mark> |
|*Appeared on*| *2022-10-17*|
|*Comments*| *15 pages, 14 figures, submitted to A&A*|
|**Abstract**| The infrared-radio correlation (IRRC) of star-forming galaxies can be used to estimate their star formation rate (SFR) based on the radio continuum luminosity at MHz-GHz frequencies. For application in future deep radio surveys, it is crucial to know whether the IRRC persists at high redshift z. Delvecchio et al. (2021) observed that the 1.4 GHz IRRC correlation of star-forming galaxies is nearly z-invariant up to z~4, but depends strongly on the stellar mass M_star. This should be taken into account for SFR calibrations based on radio luminosity. To understand the physical cause of the M_star-dependence of the IRRC and its properties at higher z, we construct a phenomenological model for galactic radio emission involving magnetic fields generated by a small-scale dynamo, a steady-state cosmic ray population, as well as observed scaling relations that reduce the number of free parameters. The best agreement between the model and the characteristics of the IRRC observed by Delvecchio et al. (2021) is found when the efficiency of the SN-driven turbulence is 5% and when saturation of the small-scale dynamo occurs once 0.5% of the kinetic energy is converted into magnetic energy. Generally, we find that the observed mass dependence of the IRRC appears as long as synchrotron emission dominates the galactic radio flux. When extrapolating the reference model to higher redshift, the free-free emission and absorption strongly affect the radio spectrum, which ultimately leads to an inversion of the M_star dependence of the IRRC at z>5. This could be tested with future deep radio observations, which will also probe the dependence of IR/radio flux ratios on galaxy orientation that is predicted by our model for high-z systems. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.07478-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.07478) | **On the underestimation of dust mass in protoplanetary disks: Effects of  disk structure and dust properties**  |
|| <mark>Yao Liu</mark>, et al. -- incl., <mark>Hendrik Linz</mark>, <mark>Thomas Henning</mark>, <mark>Mario Flock</mark>, <mark>Dafa Li</mark> |
|*Appeared on*| *2022-10-17*|
|*Comments*| *16 pages, 9 figures, accepted for publication in A&A*|
|**Abstract**| The total amount of dust grains in protoplanetary disks is one of the key properties that characterize the potential for planet formation. With (sub-)millimeter flux measurements, literature studies usually derive the dust mass using an analytic form under the assumption of optically thin emission, which may lead to substantial underestimation. In this work, we conduct a parameter study with the goal of investigating the effects of disk structure and dust properties on the underestimation through self-consistent radiative transfer models. Different dust models, scattering modes and approaches for dust settling are considered and compared. The influences of disk substructures, such as rings and crescents, on the mass derivation are investigated as well. The results indicate that the traditional analytic method can underestimate the mass by a factor of a few to hundreds, depending on the optical depth along the line of sight set mainly by the true dust mass, disk size and inclination. As an application, we perform a detailed radiative transfer modeling of the spectral energy distribution of DoAr 33, one of the observed DSHARP disks. When the DSHARP dust opacities are adopted, the most probable dust mass returned from the Bayesian analysis is roughly 7 times higher than the value given by the analytic calculation. Our study demonstrates that estimating disk dust masses from radiative transfer modeling is one solution for alleviating the problem of insufficient mass for planet formation raised in the ALMA era. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error sequence item 0: expected str instance, TexNode found</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

63  publications in the last 7 days.
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers